In [4]:
import os.path
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
from pandarallel import pandarallel
from tqdm.auto import tqdm, trange
from sklearn.metrics import auc as auc_fn
from sklearn.linear_model import LinearRegression

from utilities import deserialize_xgboost_classifier_and_get_importance_gain

Constants.

In [6]:
ROOT = '.'
OUT_FOLDER = f'{ROOT}/results'
DATA_FOLDER = f'{ROOT}/data'
FILE = f"{DATA_FOLDER}/Supp. File 1.xlsx"
N_JOBS = -1

Read results from the custom oversampling.

In [ ]:
if not os.path.exists(f'{DATA_FOLDER}/DILI_complete_oversampling_analysis2024_feature_importance_avg_gain_norm_across_folds.feather'):
    data = pd.read_feather(f'{DATA_FOLDER}/DILI_complete_oversampling_analysis2024.feather')

Extract feature importances from serialized models.

In [ ]:
if not os.path.exists(f'{DATA_FOLDER}/DILI_complete_oversampling_analysis2024_feature_importance_avg_gain_norm_across_folds.feather'):
    pandarallel.initialize(progress_bar=True, verbose=False)
    important_features = pd.DataFrame.from_dict(data['model'].parallel_apply(deserialize_xgboost_classifier_and_get_importance_gain).tolist())
    important_features.insert(0, 'dataset', data['dataset'].reset_index(drop=True), True)
    important_features.insert(1, 'descriptors', data['descriptors'].reset_index(drop=True), True)
    important_features.insert(2, 'num_stds', data['num_stds'].reset_index(drop=True), True)
    important_features.insert(3, 'samples added', data['samples added'].reset_index(drop=True), True)

In [ ]:
if not os.path.exists(f'{DATA_FOLDER}/DILI_complete_oversampling_analysis2024_feature_importance_avg_gain_norm_across_folds.feather'):
    # Ensure total gain of all features for each condition = 1
    for i in trange(0, len(important_features), 1000):
        important_features.iloc[i: i +1000, 4:] = important_features.iloc[i: i +1000, 4:].div(important_features.iloc[i: i +1000, 4:].sum(axis=1), axis=0)
    
    important_features.to_feather(f'{OUT_FOLDER}/DILI_complete_oversampling_analysis2024_feature_importance_avg_gain_norm.feather')
    # Sum across 5-folds
    important_features = important_features.groupby(['dataset', 'descriptors', 'num_stds', 'samples added'], as_index=False).sum()
    # Ensure total gain is normalized per condition (their sums are 1)
    important_features.iloc[:, 4:] = important_features.iloc[:, 4:].div(important_features.iloc[:, 4:].sum(axis=1), axis=0)
    # Save to disk.
    important_features.to_feather(f'{DATA_FOLDER}/DILI_complete_oversampling_analysis2024_feature_importance_avg_gain_norm_across_folds.feather')

In [7]:
important_features = pd.read_feather(
    f'{DATA_FOLDER}/DILI_complete_oversampling_analysis2024_feature_importance_avg_gain_norm_across_folds.feather')
# Melt and ignore NAs summed to 0
data = important_features.melt(id_vars=['dataset', 'descriptors', 'num_stds', 'samples added']).query('value != 0')

Separate feature per type.

In [8]:
bio_descs = data[data.variable.str.split('_').str[0].isin(['bip', 'btg2', 'chop', 'hmox1', 'hspa1b', 'icam1', 'p21', 'srxn1'])].variable.drop_duplicates()
chem_descs = data[~data.variable.str.startswith('ECFP_6') & ~data.variable.str.split('_').str[0].isin(['bip', 'btg2', 'chop', 'hmox1', 'hspa1b', 'icam1', 'p21', 'srxn1']) & ~data.variable.str.endswith('_WT') & ~data.variable.str.contains('^.+(?:_[ACDEFGHIKLMNPQRSTVWY]\d+[ACDEFGHIKLMNPQRSTVWY])+$')].variable.drop_duplicates()
chemfp_descs = data[data.variable.str.startswith('ECFP_6')].variable.drop_duplicates()
allchem_descs = pd.concat((chem_descs, chemfp_descs)).drop_duplicates()
biospectra_descs = data[data.variable.str.endswith('_WT') & ~data.variable.str.contains('^.+(?:_[ACDEFGHIKLMNPQRSTVWY]\d+[ACDEFGHIKLMNPQRSTVWY])+$')].variable.drop_duplicates()

bip_descs = data[data.variable.str.startswith('bip')].variable.drop_duplicates()
btg2_descs = data[data.variable.str.startswith('btg2')].variable.drop_duplicates()
chop_descs = data[data.variable.str.startswith('chop')].variable.drop_duplicates()
hmox1_descs = data[data.variable.str.startswith('hmox1')].variable.drop_duplicates()
hspa1b_descs = data[data.variable.str.startswith('hspa1b')].variable.drop_duplicates()
icam1_descs = data[data.variable.str.startswith('icam1')].variable.drop_duplicates()
p21_descs = data[data.variable.str.startswith('p21')].variable.drop_duplicates()
srxn1_descs = data[data.variable.str.startswith('srxn1')].variable.drop_duplicates()

cmax1_descs = data[data.variable.str.split('_').str[1] == 'cmax1'].variable.drop_duplicates()
cmax5_descs = data[data.variable.str.split('_').str[1] == 'cmax5'].variable.drop_duplicates()
cmax10_descs = data[data.variable.str.split('_').str[1] == 'cmax10'].variable.drop_duplicates()
cmax25_descs = data[data.variable.str.split('_').str[1] == 'cmax25'].variable.drop_duplicates()
cmax50_descs = data[data.variable.str.split('_').str[1] == 'cmax50'].variable.drop_duplicates()
cmax100_descs = data[data.variable.str.split('_').str[1] == 'cmax100'].variable.drop_duplicates()

tp24 = data[data.variable.str.split('_').str[2] == 'tp24'].variable.drop_duplicates()
tp48 = data[data.variable.str.split('_').str[2] == 'tp48'].variable.drop_duplicates()
tp72 = data[data.variable.str.split('_').str[2] == 'tp72'].variable.drop_duplicates()

Obtain the feature importance per feature type.

In [9]:
features = [('bio', bio_descs), ('allchem', allchem_descs), ('chem', chem_descs), ('chemfp', chemfp_descs), ('biospectra', biospectra_descs),
            ('bip', bip_descs), ('btg2', btg2_descs), ('chop', chop_descs), ('hmox1', hmox1_descs), ('hspa1b', hspa1b_descs), ('icam1', icam1_descs), ('p21', p21_descs), ('srxn1', srxn1_descs),
            ('cmax1', cmax1_descs), ('cmax5', cmax5_descs), ('cmax10', cmax10_descs), ('cmax25', cmax25_descs), ('cmax50', cmax50_descs), ('cmax100', cmax100_descs),
            ('24h', tp24), ('48h', tp48), ('72h', tp72)]

table1 = []
for dataset in tqdm(['All', 'GFP', 'AnV', 'PI' , 'AnV-PI', 'GFP-AnV-PI',
                     'IntegratedIntensity', 'MeanIntensity', 'NormCounts']):
    for descriptors in ['bio', 'biochem', 'biospectra']:
        for num_stds in range(1, 6):
            x = data.query(f'dataset == "{dataset}" and descriptors == "{descriptors}" and num_stds == {num_stds}', engine='python')
            results1 = {}
            results2 = {}
            for feature_name, feature_list in features:
                values = x.query(f'variable in @feature_list', engine='python').groupby('samples added', as_index=False).sum()[['samples added', 'value']]
                if values.empty:
                    results1[feature_name] = None
                    results2[feature_name] = None
                else:
                    auc = auc_fn(values['samples added']/250, values['value'])
                    slope = LinearRegression().fit(values['samples added'].values.reshape(-1, 1), values['value']).coef_.item()
                    results1[feature_name] = auc
                    results2[feature_name] = slope
            results1 = pd.DataFrame([results1])
            results1.insert(0, 'dataset', dataset)
            results1.insert(1, 'descriptors', descriptors)
            results1.insert(2, 'num_stds', num_stds)
            results2 = pd.DataFrame([results2])
            results2.insert(0, 'dataset', dataset)
            results2.insert(1, 'descriptors', descriptors)
            results2.insert(2, 'num_stds', num_stds)
            table1.append(results1)

table1 = pd.concat(table1)

  0%|          | 0/9 [00:00<?, ?it/s]

Obtain Figure 4A, B, and C.

In [10]:
table1.query('dataset == "All" and num_stds == 1').drop(columns=['dataset', 'num_stds']).groupby('descriptors').agg('mean').round(4).T

descriptors,bio,biochem,biospectra
bio,1.0000,0.9859,0.8531
allchem,NaN,0.0141,0.0042
chem,NaN,0.0053,0.0009
chemfp,NaN,0.0088,0.0034
biospectra,NaN,NaN,0.1425
bip,0.0896,0.0892,0.0701
btg2,0.1146,0.1099,0.0975
chop,0.1212,0.1200,0.1115
hmox1,0.1116,0.1123,0.0950
hspa1b,0.0717,0.0686,0.0569


Obtain first part of Figure 4D. 

In [11]:
table3 = []
dataset  = 'All'
for descriptors in tqdm(['bio', 'biochem', 'biospectra']):
    for num_stds in range(1, 6):
        x = data.query(f'dataset == "{dataset}" and descriptors == "{descriptors}" and num_stds == {num_stds}', engine='python')
        results1 = {}
        results2 = {}
        for feature_name in chem_descs:
            values = x.query(f'variable == "{feature_name}"', engine='python')[['samples added', 'value']]
            if values.empty or values.shape[0] == 1:
                results1[feature_name] = None
            else:
                auc = auc_fn(values['samples added']/250, values['value'])
                slope = LinearRegression().fit(values['samples added'].values.reshape(-1, 1), values['value']).coef_.item()
                results1[feature_name] = auc
                results2[feature_name] = slope
        results1 = pd.DataFrame([results1])
        results1.insert(0, 'dataset', dataset)
        results1.insert(1, 'descriptors', descriptors)
        results1.insert(2, 'num_stds', num_stds)
        table3.append(results1)
        results2 = pd.DataFrame([results2])
        results2.insert(0, 'dataset', dataset)
        results2.insert(1, 'descriptors', descriptors)
        results2.insert(2, 'num_stds', num_stds)

table3 = pd.concat(table3)

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
table3.query('dataset == "All" and num_stds == 1').drop(columns=['dataset', 'num_stds']).groupby('descriptors').agg('mean').round(4).T.sort_values('biochem', ascending=False)

descriptors,bio,biochem,biospectra
ES_Sum_sOH,NaN,0.0015,0.0001
ALogP,NaN,0.0011,0.0002
ES_Sum_sssCH,NaN,0.0009,0.0001
ES_Sum_dssC,NaN,0.0005,NaN
ES_Sum_ssCH2,NaN,0.0004,0.0
CHI_V_3_C,NaN,0.0004,NaN
ES_Sum_ddsN,NaN,0.0004,0.0003
ES_Sum_sssN,NaN,0.0002,NaN
H_Count,NaN,0.0002,NaN
ES_Sum_dO,NaN,0.0002,0.0


Obtain the second part of Figure 4D.

In [13]:
table5 = []
dataset  = 'All'
for descriptors in tqdm(['bio', 'biochem', 'biospectra']):
    for num_stds in range(1, 6):
        x = data.query(f'dataset == "{dataset}" and descriptors == "{descriptors}" and num_stds == {num_stds}', engine='python')
        results1 = {}
        results2 = {}
        for feature_name in chemfp_descs:
            values = x.query(f'variable == "{feature_name}"', engine='python')[['samples added', 'value']]
            if values.empty or values.shape[0] == 1:
                results1[feature_name] = None
            else:
                auc = auc_fn(values['samples added']/250, values['value'])
                slope = LinearRegression().fit(values['samples added'].values.reshape(-1, 1), values['value']).coef_.item()
                results1[feature_name] = auc
                results2[feature_name] = slope
        results1 = pd.DataFrame([results1])
        results1.insert(0, 'dataset', dataset)
        results1.insert(1, 'descriptors', descriptors)
        results1.insert(2, 'num_stds', num_stds)
        table5.append(results1)
        results2 = pd.DataFrame([results2])
        results2.insert(0, 'dataset', dataset)
        results2.insert(1, 'descriptors', descriptors)
        results2.insert(2, 'num_stds', num_stds)

table5 = pd.concat(table5)

  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
table5.query('dataset == "All" and num_stds == 1').drop(columns=['dataset', 'num_stds']).groupby('descriptors').agg(
    'mean').round(4).T.sort_values('biochem', ascending=False)

descriptors,bio,biochem,biospectra
ECFP_6_684,NaN,0.0027,0.0009
ECFP_6_594,NaN,0.0024,0.0004
ECFP_6_624,NaN,0.0008,0.0006
ECFP_6_1584,NaN,0.0008,0.0001
ECFP_6_299,NaN,0.0005,NaN
...,...,...,...
ECFP_6_1362,NaN,NaN,NaN
ECFP_6_1663,NaN,NaN,NaN
ECFP_6_168,NaN,NaN,NaN
ECFP_6_283,NaN,NaN,NaN


Obtain Figure 4E.

In [15]:
proteins = pd.read_csv(f'{DATA_FOLDER}/05.7_combined_set_protein_targets.tsv.xz', sep='\t')
human_proteins = proteins.query('Organism == "Homo sapiens (Human)"')
human_data = data.query('variable in @human_proteins.target_id')
data_organism = data.merge(proteins[['target_id', 'Organism', 'Classification']], left_on='variable', right_on='target_id')

In [16]:
table11 = []
dataset  = 'All'
for descriptors in tqdm(['bio', 'biochem', 'biospectra']):
    for num_stds in range(1, 6):
        x = data_organism.query(f'dataset == "{dataset}" and descriptors == "{descriptors}" and num_stds == {num_stds}', engine='python')
        results1 = {}
        results2 = {}
        for organism in data_organism.Organism.unique():
            tmp_org_proteins = data_organism.query('Organism  == "' + organism + '"').target_id
            values = x.query(f'variable in @tmp_org_proteins', engine='python').groupby('samples added', as_index=False).sum()[['samples added', 'value']]
            if values.empty or values.shape[0] == 1:
                results1[organism] = None
            else:
                auc = auc_fn(values['samples added']/250, values['value'])
                slope = LinearRegression().fit(values['samples added'].values.reshape(-1, 1), values['value']).coef_.item()
                results1[organism] = auc
                results2[organism] = slope
        results1 = pd.DataFrame([results1])
        results1.insert(0, 'dataset', dataset)
        results1.insert(1, 'descriptors', descriptors)
        results1.insert(2, 'num_stds', num_stds)
        table11.append(results1)
        results2 = pd.DataFrame([results2])
        results2.insert(0, 'dataset', dataset)
        results2.insert(1, 'descriptors', descriptors)
        results2.insert(2, 'num_stds', num_stds)


table11 = pd.concat(table11)

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
table11.query('dataset == "All" and num_stds == 1').drop(columns=['dataset', 'num_stds']).groupby('descriptors').agg(
    'mean').round(4).T.sort_values('biospectra', ascending=False)

descriptors,bio,biochem,biospectra
Homo sapiens (Human),NaN,NaN,0.0607
Escherichia coli (strain K12),NaN,NaN,0.0188
Rattus norvegicus (Rat),NaN,NaN,0.0115
Mus musculus (Mouse),NaN,NaN,0.0112
Bacillus amyloliquefaciens (Bacillus velezensis),NaN,NaN,0.0052
...,...,...,...
Flaveria bidentis (Coastal plain yellowtops) (Ethulia bidentis),NaN,NaN,NaN
Staphylococcus saprophyticus subsp. saprophyticus (strain ATCC 15305 / DSM 20229 / NCIMB 8711 / NCTC 7292 / S-41),NaN,NaN,NaN
Norovirus Hu/GI/10360/2010/VNM,NaN,NaN,NaN
Emericella nidulans (Aspergillus nidulans),NaN,NaN,NaN


Obtain Figure 4F.

In [18]:
table9 = []
dataset  = 'All'
for descriptors in tqdm(['bio', 'biochem', 'biospectra']):
    for num_stds in range(1, 6):
        x = human_data.query(f'dataset == "{dataset}" and descriptors == "{descriptors}" and num_stds == {num_stds}', engine='python')
        results1 = {}
        results2 = {}
        for feature_name in human_data.variable.unique():
            values = x.query(f'variable == "{feature_name}"', engine='python')[['samples added', 'value']]
            if values.empty or values.shape[0] == 1:
                results1[feature_name] = None
            else:
                auc = auc_fn(values['samples added']/250, values['value'])
                slope = LinearRegression().fit(values['samples added'].values.reshape(-1, 1), values['value']).coef_.item()
                results1[feature_name] = auc
                results2[feature_name] = slope
        results1 = pd.DataFrame([results1])
        results1.insert(0, 'dataset', dataset)
        results1.insert(1, 'descriptors', descriptors)
        results1.insert(2, 'num_stds', num_stds)
        table9.append(results1)
        results2 = pd.DataFrame([results2])
        results2.insert(0, 'dataset', dataset)
        results2.insert(1, 'descriptors', descriptors)
        results2.insert(2, 'num_stds', num_stds)

table9 = pd.concat(table9)

  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
table9.query('dataset == "All" and num_stds == 1').drop(columns=['dataset', 'num_stds']).groupby('descriptors').agg(
    'mean').round(4).T.sort_values('biospectra', ascending=False)

descriptors,bio,biochem,biospectra
Q9BTE7_WT,NaN,NaN,0.0029
Q6R6M4_WT,NaN,NaN,0.0027
Q01469_WT,NaN,NaN,0.0025
P62081_WT,NaN,NaN,0.0019
P54577_WT,NaN,NaN,0.0019
...,...,...,...
O75388_WT,NaN,NaN,NaN
P07203_WT,NaN,NaN,NaN
P16455_WT,NaN,NaN,NaN
Q15843_WT,NaN,NaN,NaN


Obtain Supp. Tables 2 to 6.

In [20]:
ttperfs = pd.read_feather(fr'{OUT_FOLDER}/Analysis_DILI_num_samples_to_high_specificity.feather')

In [21]:
ttperfs.query('num_stds == 1')

baseline  #samples Spe > 0.70  \
dataset    num_stds descriptor metric                                  
All        1        bio        MCC     0.062779                   30   
                               BAcc    0.519000                   30   
                               Sen     0.991333                   30   
                               Spe     0.046667                   30   
                    biochem    MCC     0.048075                   30   
...                                         ...                  ...   
NumObjects 1        biochem    Spe     0.082222                   30   
                    biospectra MCC     0.065587                   60   
                               BAcc    0.520444                   60   
                               Sen     0.992000                   60   
                               Spe     0.048889                   60   

                                       perf at Spe > 0.70  \
dataset    num_stds descriptor metric                       
All        1        bio        MCC               0.734304   
                               BAcc              0.838667   
                               Sen               0.968000   
                               Spe               0.709333   
                    biochem    MCC               0.724414   
...                                                   ...   
NumObjects 1        biochem    Spe               0.722519   
                    biospectra MCC               0.597147   
                               BAcc              0.793250   
                               Sen               0.844667   
                               Spe               0.741833   

                                       #samples Spe > 0.80  \
dataset    num_stds descriptor metric                        
All        1        bio        MCC                      40   
                               BAcc                     40   
                               Sen                      40   
                               Spe                      40   
                    biochem    MCC                      40   
...                                                    ...   
NumObjects 1        biochem    Spe                      40   
                    biospectra MCC                      70   
                               BAcc                     70   
                               Sen                      70   
                               Spe                      70   

                                       perf at Spe > 0.80  \
dataset    num_stds descriptor metric                       
All        1        bio        MCC               0.792534   
                               BAcc              0.882990   
                               Sen               0.954667   
                               Spe               0.811313   
                    biochem    MCC               0.790802   
...                                                   ...   
NumObjects 1        biochem    Spe               0.830909   
                    biospectra MCC               0.652399   
                               BAcc              0.824423   
                               Sen               0.826667   
                               Spe               0.822179   

                                       #samples Spe > 0.90  \
dataset    num_stds descriptor metric                        
All        1        bio        MCC                      80   
                               BAcc                     80   
                               Sen                      80   
                               Spe                      80   
                    biochem    MCC                      80   
...                                                    ...   
NumObjects 1        biochem    Spe                      60   
                    biospectra MCC                      90   
                               BAcc                     90   
                         

In [22]:
ttperfs.query('num_stds == 2')

baseline  #samples Spe > 0.70  \
dataset    num_stds descriptor metric                                  
All        2        bio        MCC     0.062779                   40   
                               BAcc    0.519000                   40   
                               Sen     0.991333                   40   
                               Spe     0.046667                   40   
                    biochem    MCC     0.048075                   40   
...                                         ...                  ...   
NumObjects 2        biochem    Spe     0.074444                   30   
                    biospectra MCC     0.062884                   60   
                               BAcc    0.519111                   60   
                               Sen     0.992667                   60   
                               Spe     0.045556                   60   

                                       perf at Spe > 0.70  \
dataset    num_stds descriptor metric                       
All        2        bio        MCC               0.764027   
                               BAcc              0.861626   
                               Sen               0.961333   
                               Spe               0.761919   
                    biochem    MCC               0.756279   
...                                                   ...   
NumObjects 2        biochem    Spe               0.703704   
                    biospectra MCC               0.616787   
                               BAcc              0.803472   
                               Sen               0.850667   
                               Spe               0.756278   

                                       #samples Spe > 0.80  \
dataset    num_stds descriptor metric                        
All        2        bio        MCC                      50   
                               BAcc                     50   
                               Sen                      50   
                               Spe                      50   
                    biochem    MCC                      50   
...                                                    ...   
NumObjects 2        biochem    Spe                      50   
                    biospectra MCC                      70   
                               BAcc                     70   
                               Sen                      70   
                               Spe                      70   

                                       perf at Spe > 0.80  \
dataset    num_stds descriptor metric                       
All        2        bio        MCC               0.803930   
                               BAcc              0.892659   
                               Sen               0.950667   
                               Spe               0.834652   
                    biochem    MCC               0.810971   
...                                                   ...   
NumObjects 2        biochem    Spe               0.865275   
                    biospectra MCC               0.645934   
                               BAcc              0.819932   
                               Sen               0.822000   
                               Spe               0.817865   

                                       #samples Spe > 0.90  \
dataset    num_stds descriptor metric                        
All        2        bio        MCC                      70   
                               BAcc                     70   
                               Sen                      70   
                               Spe                      70   
                    biochem    MCC                      70   
...                                                    ...   
NumObjects 2        biochem    Spe                      70   
                    biospectra MCC                      90   
                               BAcc                     90   
                         

In [23]:
ttperfs.query('num_stds == 3')

baseline  #samples Spe > 0.70  \
dataset    num_stds descriptor metric                                  
All        3        bio        MCC     0.062779                   40   
                               BAcc    0.519000                   40   
                               Sen     0.991333                   40   
                               Spe     0.046667                   40   
                    biochem    MCC     0.048075                   40   
...                                         ...                  ...   
NumObjects 3        biochem    Spe     0.112222                   40   
                    biospectra MCC     0.078894                   60   
                               BAcc    0.525000                   60   
                               Sen     0.990000                   60   
                               Spe     0.060000                   60   

                                       perf at Spe > 0.70  \
dataset    num_stds descriptor metric                       
All        3        bio        MCC               0.756405   
                               BAcc              0.855909   
                               Sen               0.966667   
                               Spe               0.745152   
                    biochem    MCC               0.741977   
...                                                   ...   
NumObjects 3        biochem    Spe               0.757677   
                    biospectra MCC               0.638946   
                               BAcc              0.814139   
                               Sen               0.863333   
                               Spe               0.764944   

                                       #samples Spe > 0.80  \
dataset    num_stds descriptor metric                        
All        3        bio        MCC                      60   
                               BAcc                     60   
                               Sen                      60   
                               Spe                      60   
                    biochem    MCC                      60   
...                                                    ...   
NumObjects 3        biochem    Spe                      60   
                    biospectra MCC                      70   
                               BAcc                     70   
                               Sen                      70   
                               Spe                      70   

                                       perf at Spe > 0.80  \
dataset    num_stds descriptor metric                       
All        3        bio        MCC               0.812878   
                               BAcc              0.898472   
                               Sen               0.952000   
                               Spe               0.844944   
                    biochem    MCC               0.821743   
...                                                   ...   
NumObjects 3        biochem    Spe               0.858000   
                    biospectra MCC               0.677174   
                               BAcc              0.836020   
                               Sen               0.840667   
                               Spe               0.831373   

                                       #samples Spe > 0.90  \
dataset    num_stds descriptor metric                        
All        3        bio        MCC                      80   
                               BAcc                     80   
                               Sen                      80   
                               Spe                      80   
                    biochem    MCC                      80   
...                                                    ...   
NumObjects 3        biochem    Spe                      70   
                    biospectra MCC                     100   
                               BAcc                    100   
                         

In [24]:
ttperfs.query('num_stds == 4')

baseline  #samples Spe > 0.70  \
dataset    num_stds descriptor metric                                  
All        4        bio        MCC     0.062779                   40   
                               BAcc    0.519000                   40   
                               Sen     0.991333                   40   
                               Spe     0.046667                   40   
                    biochem    MCC     0.048075                   40   
...                                         ...                  ...   
NumObjects 4        biochem    Spe     0.074444                   40   
                    biospectra MCC     0.074795                   50   
                               BAcc    0.522222                   50   
                               Sen     0.986667                   50   
                               Spe     0.057778                   50   

                                       perf at Spe > 0.70  \
dataset    num_stds descriptor metric                       
All        4        bio        MCC               0.740390   
                               BAcc              0.846081   
                               Sen               0.966000   
                               Spe               0.726162   
                    biochem    MCC               0.721714   
...                                                   ...   
NumObjects 4        biochem    Spe               0.746162   
                    biospectra MCC               0.639147   
                               BAcc              0.808256   
                               Sen               0.896000   
                               Spe               0.720513   

                                       #samples Spe > 0.80  \
dataset    num_stds descriptor metric                        
All        4        bio        MCC                      60   
                               BAcc                     60   
                               Sen                      60   
                               Spe                      60   
                    biochem    MCC                      60   
...                                                    ...   
NumObjects 4        biochem    Spe                      50   
                    biospectra MCC                      70   
                               BAcc                     70   
                               Sen                      70   
                               Spe                      70   

                                       perf at Spe > 0.80  \
dataset    num_stds descriptor metric                       
All        4        bio        MCC               0.801149   
                               BAcc              0.892111   
                               Sen               0.950667   
                               Spe               0.833556   
                    biochem    MCC               0.805938   
...                                                   ...   
NumObjects 4        biochem    Spe               0.801905   
                    biospectra MCC               0.697724   
                               BAcc              0.846416   
                               Sen               0.856667   
                               Spe               0.836166   

                                       #samples Spe > 0.90  \
dataset    num_stds descriptor metric                        
All        4        bio        MCC                      90   
                               BAcc                     90   
                               Sen                      90   
                               Spe                      90   
                    biochem    MCC                      80   
...                                                    ...   
NumObjects 4        biochem    Spe                      90   
                    biospectra MCC                     100   
                               BAcc                    100   
                         

In [25]:
ttperfs.query('num_stds == 5')

baseline  #samples Spe > 0.70  \
dataset    num_stds descriptor metric                                  
All        5        bio        MCC     0.062779                   40   
                               BAcc    0.519000                   40   
                               Sen     0.991333                   40   
                               Spe     0.046667                   40   
                    biochem    MCC     0.048075                   40   
...                                         ...                  ...   
NumObjects 5        biochem    Spe     0.063333                   40   
                    biospectra MCC     0.048446                   50   
                               BAcc    0.514556                   50   
                               Sen     0.984667                   50   
                               Spe     0.044444                   50   

                                       perf at Spe > 0.70  \
dataset    num_stds descriptor metric                       
All        5        bio        MCC               0.737053   
                               BAcc              0.843970   
                               Sen               0.964000   
                               Spe               0.723939   
                    biochem    MCC               0.736424   
...                                                   ...   
NumObjects 5        biochem    Spe               0.746061   
                    biospectra MCC               0.613655   
                               BAcc              0.797875   
                               Sen               0.880000   
                               Spe               0.715751   

                                       #samples Spe > 0.80  \
dataset    num_stds descriptor metric                        
All        5        bio        MCC                      60   
                               BAcc                     60   
                               Sen                      60   
                               Spe                      60   
                    biochem    MCC                      60   
...                                                    ...   
NumObjects 5        biochem    Spe                      60   
                    biospectra MCC                      70   
                               BAcc                     70   
                               Sen                      70   
                               Spe                      70   

                                       perf at Spe > 0.80  \
dataset    num_stds descriptor metric                       
All        5        bio        MCC               0.792553   
                               BAcc              0.886556   
                               Sen               0.952667   
                               Spe               0.820444   
                    biochem    MCC               0.786188   
...                                                   ...   
NumObjects 5        biochem    Spe               0.818722   
                    biospectra MCC               0.693431   
                               BAcc              0.842582   
                               Sen               0.876667   
                               Spe               0.808497   

                                       #samples Spe > 0.90  \
dataset    num_stds descriptor metric                        
All        5        bio        MCC                     100   
                               BAcc                    100   
                               Sen                     100   
                               Spe                     100   
                    biochem    MCC                     100   
...                                                    ...   
NumObjects 5        biochem    Spe                     100   
                    biospectra MCC                     110   
                               BAcc                    110   
                         

Obtain Supp. Tables 7 to 10.

In [26]:
importances = ((tmp := data.query('num_stds == 1 and descriptors == "bio" and dataset == "All" and variable in @bio_descs'))
                           .assign(category=tmp.variable
                                               .str.split('_')
                                               .str[:3]
                                               .str.join ('_'))
                           .groupby(['category', 'samples added'])
                           .value.sum()
               )

importances_ = []
for name, group in importances.reset_index().groupby('category'):
    auc = auc_fn(group['samples added']/250, group['value'])
    importances_.append(pd.DataFrame([{'category': name, 'value': auc}]))

importances = (pd.concat(importances_, ignore_index=True)
                 .set_index('category'))
importances /= importances.value.sum()

importances = importances.reset_index()
importances = (importances.assign(reporter=importances.category.str.split('_').str[0],
                                  concentration=importances.category.str.split('_').str[1],
                                  timepoint=importances.category.str.split('_').str[2])
               ).drop(columns='category')

importances = importances.pivot(index=['reporter', 'timepoint'], columns='concentration', values='value') * 100
importances

concentration          cmax1    cmax10   cmax100    cmax25     cmax5    cmax50
reporter timepoint                                                            
bip      tp24       0.705715  0.345843  0.216903  0.112859  0.417224  0.102223
         tp48       0.324726  1.654604  0.125684  0.144477  0.307734  0.159356
         tp72       0.258872  0.290374  2.862382  0.150929  0.431828  0.376236
btg2     tp24       1.199834  0.540015  0.134655  0.115066  2.773389  0.745783
         tp48       0.199320  0.843821  0.092102  0.241312  2.678558  0.181152
         tp72       0.813219  0.189861  0.119470  0.058446  0.410295  0.105886
chop     tp24       0.312875  0.223616  1.524237  0.523871  0.239511  0.298709
         tp48       0.096797  0.099977  1.867967  0.217607  0.487782  2.375727
         tp72       0.067532  0.673431  1.079659  0.444545  1.500177  0.068180
hmox1    tp24       0.178511  1.141315  1.034693  0.140493  0.071116  0.114615
         tp48       0.254292  0.147990  0.241638  0.126019  4.987010  0.067353
         tp72       0.998343  0.054134  1.101262  0.055749  0.413207  0.060134
hspa1b   tp24       0.149025  0.365433  2.440405  0.424624  0.102203  0.146549
         tp48       0.178262  0.700808  0.372027  0.163405  0.778841  0.259291
         tp72       0.058428  0.086747  0.400388  0.113242  0.249778  0.261841
icam1    tp48       0.461091  1.230662  0.968427  0.529176  0.189328  0.372197
         tp72       0.567937  4.588135  0.191737  0.094693  3.301522  0.107916
p21      tp24       0.989187  4.135165  0.416414  0.184838  0.052095  0.690360
         tp48       0.288079  0.208255  0.134240  0.170332  0.135627  1.613405
         tp72       0.068558  0.077518  3.296753  0.036649  3.559519  0.063303
srxn1    tp24       0.251003  0.180499  0.701016  0.440458  0.181545  9.249447
         tp48       0.136826  0.180417  1.787575  0.419083  0.362565  2.201305
         tp72       0.333469  1.685926  1.065894  0.605345  0.175980  0.347007

Supp. Table 7.

In [27]:
importances.reset_index(level=1).iloc[:, 1:].reset_index().groupby('reporter').agg('sum').sum(axis=1).rename('total feature importance (%)').to_frame()

,total feature importance (%)
reporter,
bip,8.987968
btg2,11.442186
chop,12.102201
hmox1,11.187872
hspa1b,7.251298
icam1,12.602821
p21,16.120295
srxn1,20.305358


Supp. Table 8.

In [28]:
importances.sum(axis=1).rename('importnace').reset_index().pivot(index='reporter', columns='timepoint')

importnace                    
timepoint       tp24      tp48      tp72
reporter                                
bip         1.900767  2.716581  4.370621
btg2        5.508742  4.236266  1.697178
chop        3.122819  5.145857  3.833525
hmox1       2.680743  5.824300  2.682829
hspa1b      3.628240  2.452633  1.170425
icam1            NaN  3.750881  8.851940
p21         6.468058  2.549938  7.102299
srxn1      11.003968  5.087770  4.213620

Supp. Table 9.

In [29]:
importances.reset_index(level=1).iloc[:, 1:].reset_index().groupby('reporter').agg('sum')

concentration,cmax1,cmax10,cmax100,cmax25,cmax5,cmax50
reporter,,,,,,
bip,1.289312,2.290821,3.204969,0.408265,1.156786,0.637815
btg2,2.212373,1.573697,0.346227,0.414824,5.862242,1.032822
chop,0.477204,0.997025,4.471863,1.186023,2.227470,2.742616
hmox1,1.431145,1.343439,2.377592,0.322261,5.471333,0.242102
hspa1b,0.385715,1.152988,3.212820,0.701271,1.130822,0.667681
icam1,1.029028,5.818796,1.160164,0.623869,3.490850,0.480114
p21,1.345824,4.420938,3.847407,0.391818,3.747240,2.367069
srxn1,0.721298,2.046842,3.554484,1.464885,0.720089,11.797759


Obtain Supp. Table 11.

In [30]:
# Importances per maker type
marker_importances = ((tmp := data.query('num_stds == 1 and descriptors == "bio" and dataset == "All" and variable in @bio_descs'))
                           .assign(category=tmp.variable
                                               .str.split('_')
                                               .str[-1])
                           .groupby(['category', 'samples added'])
                           .value.sum()
               )

importances_ = []
for name, group in marker_importances.reset_index().groupby('category'):
    auc = auc_fn(group['samples added']/250, group['value'])
    importances_.append(pd.DataFrame([{'category': name, 'value': auc}]))

marker_importances = (pd.concat(importances_, ignore_index=True)
                 .set_index('category'))
marker_importances /= marker_importances.value.sum()

marker_importances = marker_importances.reset_index()
marker_importances

,category,value
0,AnnexinPos,0.174250
1,CytoplasmNucleiIntegratedIntensityImageGfp,0.059486
2,CytoplasmNucleiMeanIntensityImageGfp,0.073349
3,DmsoIntegratedIntensityPlateLognorm,0.034517
4,DmsoMeanCellCounts,0.034757
5,GfpDiff1i,0.010474
6,GfpDiff2i,0.005004
7,GfpDiff3i,0.001908
8,GfpDiff4i,0.005373
9,GfpNeg1i,0.008640
